In [3]:
from keras.models import Sequential, Model
from keras import Input, layers

seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

seq_model.summary()

input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

model = Model(input_tensor, output_tensor)

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_8 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_9 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_10 (Dense)             (None, 32)                2080      
______________________________

In [6]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
import numpy as np
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

model.fit(x_train, y_train, epochs=10, batch_size=128)

score = model.evaluate(x_train, y_train)
print(score)

Epoch 1/10
8/8 [==============================] - 0s 870us/step - loss: 43.0607
Epoch 2/10
8/8 [==============================] - 0s 875us/step - loss: 48.2898
Epoch 3/10
8/8 [==============================] - 0s 878us/step - loss: 52.8972
Epoch 4/10
8/8 [==============================] - 0s 873us/step - loss: 57.5671
Epoch 5/10
8/8 [==============================] - 0s 998us/step - loss: 62.1977
Epoch 6/10
8/8 [==============================] - 0s 1ms/step - loss: 66.2559
Epoch 7/10
8/8 [==============================] - 0s 1ms/step - loss: 71.1632
Epoch 8/10
8/8 [==============================] - 0s 876us/step - loss: 76.2351
Epoch 9/10
8/8 [==============================] - 0s 1ms/step - loss: 81.6900
Epoch 10/10
32/32 [==============================] - 0s 467us/step - loss: 89.9956
89.99559783935547


In [8]:
from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), dtype='int32', name='text')
embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [9]:
import numpy as np
from keras.utils import to_categorical

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))

answers = np.random.randint(0, answer_vocabulary_size, size=num_samples)
answers = to_categorical(answers)

model.fit({'text' : text, 'question' : question}, answers, epochs=10, batch_size=128)
# model.fit([text, question], answers, epochs=10, batch_size=128)

Epoch 1/10
8/8 [==============================] - 0s 55ms/step - loss: 6.2146 - acc: 0.0020
Epoch 2/10
8/8 [==============================] - 0s 53ms/step - loss: 6.1969 - acc: 0.0360
Epoch 3/10
8/8 [==============================] - 0s 51ms/step - loss: 6.1397 - acc: 0.0050
Epoch 4/10
8/8 [==============================] - 0s 48ms/step - loss: 6.0539 - acc: 0.0050
Epoch 5/10
8/8 [==============================] - 0s 48ms/step - loss: 5.9914 - acc: 0.0090
Epoch 6/10
8/8 [==============================] - 0s 47ms/step - loss: 5.9209 - acc: 0.0100
Epoch 7/10
8/8 [==============================] - 0s 50ms/step - loss: 5.8407 - acc: 0.0140
Epoch 8/10
8/8 [==============================] - 0s 47ms/step - loss: 5.7554 - acc: 0.0170
Epoch 9/10
8/8 [==============================] - 0s 50ms/step - loss: 5.6623 - acc: 0.0210
Epoch 10/10
8/8 [==============================] - 0s 47ms/step - loss: 5.5970 - acc: 0.0380


In [12]:
from keras import layers
from keras import Input
from keras.models import Model
vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(vocabulary_size, 256)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups, activation='softmax', name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])

In [13]:
model.compile(optimizer='rmsprop', loss={'age' : 'mse', 'income' : 'categorical_crossentropy', 'gender' : 'binary_crossentropy'})
# model.compile(optimizer='rmsprop', loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'])

In [14]:
model.compile(optimizer='rmsprop', loss={'age' : 'mse', 'income' : 'categorical_crossentropy', 'gender' : 'binary_crossentropy'},
             loss_weights={'age' : 0.25, 'income' : 1., 'gender' : 10.})

In [15]:
model.fit(posts, [age_targets, income_targets, gender_targets], epochs=10, batch_size=64)

NameError: name 'posts' is not defined

In [16]:
#Inception module
from keras import layers

branch_a = layers.Conv2D(128, 1, activation='relu', strides=2)(x)

branch_b = layers.Conv2D(128, 1, activation='relu')(x)
branch_b = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_b)

branch_c = layers.AveragePooling2D(2, strides=2)(x)
branch_c = layers.Conv2D(128, 3, activation='relu')(branch_c)

branch_d = layers.Conv2D(128, 1, activation='relu')(x)
branch_d = layers.Conv2D(128, 3, activation='relu')(branch_d)
branch_d = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_d)

output = layers.concatenate([branch_a, branch_b, branch_c, branch_d], axis=-1)

ValueError: Input 0 of layer conv2d is incompatible with the layer: : expected min_ndim=4, found ndim=2. Full shape received: [None, 128]

In [18]:
#Residual connection when size of feature maps of x and y are same
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.add([y, x])

#Residual connection when size of feature maps of x and y are different
y = layers.Conv2D(128, 2, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 2, activation='relu', padding='same')(y)
y = layers.MaxPooling2D(2, strides=2)(y)
residual = layers.Conv2D(128, 1, strides=2, padding='same')(x)
y = layers.add([y, residual])

ValueError: Input 0 of layer conv2d_1 is incompatible with the layer: : expected min_ndim=4, found ndim=2. Full shape received: [None, 128]

In [19]:
from keras import layers
from keras import Input
from keras.models import Model

lstm = layers.LSTM(32)
left_input = Input(shape=(None, 128))
left_output = lstm(left_input)

right_input = Input(shape=(None, 128))
right_output = lstm(right_input)

merged = layers.concatenate([left_output, right_output], axis=-1)
predictions = layers.Dense(1, activation='sigmoid')(merged)

model = Model([left_input, right_input], predictions)
model.fit([left_data, right_data], targets)

NameError: name 'left_data' is not defined

In [20]:
from keras import layers
from keras import applications
from keras import Input

xception_base = applications.Xception(weights=None, include_top=False)

left_input = Input(shape=(250, 250, 3))
right_input = Input(shape=(250, 250, 3))
left_features = xception_base(left_input)
right_features = xception_base(right_input)

merged_features = layers.concatenate([left_features, right_features], axis=-1)